## Building SVM Classifier

In [131]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC 
from sklearn.svm import SVC

import sys
# sys.path.append("/Users/Work/Developer/interpretDL/interprettensor")
root_logdir = "./tf_logs"

# To plot pretty figures
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

    
def remove_label(features, labels, label="MCI"):
    labels = pd.Series(fused_labels)
    non_samples = labels != label

    stripped_features = features[non_samples]
    stripped_labels = labels[non_samples]

    return stripped_features, stripped_labels

def get_split(features, labels):
    features = np.array(features)
    # The train set will have equal amounts of each target class
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(features, labels):
        X_train = features[train_index]
        y_train = labels[train_index]
        X_test = features[test_index]
        y_test = labels[test_index]
        
        yield X_train, y_train, X_test, y_test


tf.__version__

'1.13.1'

In [123]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# Helper Functions

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

######### Taken from sklearn #######
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=[8,8])
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


class AttributeRemover(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return # Doesn't do anything
    
    def transform(self, X, y=None):
        return X.drop(columns=self.attribute_names)

# class OverSampler(BaseEstimator, TransformerMixin):
#     """
#     Returns a copy of matrix with attributes removed
#     """
#     def __init__(self, random_state=42):
#         self.smote = SMOTE(random_state=random_state)
    
#     def fit(self, X, y=None):
#         self.smote.fit(X,y)
#         return self
    
#     def transform(self, X, y=None):
#         return self.smote.resample(X,y)

In [164]:
# # Getting MNIST data
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# # Converting to float and normalizing
# X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
# X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
# y_train = y_train.astype(np.int32)
# y_test = y_test.astype(np.int32)

# # Separating out validation set
# X_valid, X_train = X_train[:5000], X_train[5000:]
# y_valid, y_train = y_train[:5000], y_train[5000:]

# X_train.shape

# svm_clf = Pipeline([
#     ("scaler", StandardScaler()),
#     ("SVM", SVC(kernel= "rbf", gamma= 10, C=0.01))
# ])

# %time svm_clf.fit(X_train[:500], y_train[:500])
# svm_clf.score(X_valid[:100], y_valid[:100])

## Testing on ADNI data

In [191]:
filename = "ICV_ADNI.csv"
raw_data = pd.read_csv(filename)
print(raw_data.info())


label_col = "DX_bl"
non_feature_cols = ["PTID", "scandate", "ICV",label_col]

# Getting all the columns related to surface area
thickness_features = [x for x in features.columns if "SA" in x ]

raw_features = AttributeRemover(attribute_names= non_feature_cols + thickness_features).transform(raw_data)
raw_labels = raw_data[label_col].copy()

raw_features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Columns: 300 entries, PTID to DX_bl
dtypes: float64(149), int64(148), object(3)
memory usage: 335.2+ KB
None


,G_and_S_frontomargin_TH_lh,G_and_S_occipital_inf_TH_lh,G_and_S_paracentral_TH_lh,G_and_S_subcentral_TH_lh,G_and_S_transv_frontopol_TH_lh,G_and_S_cingul-Ant_TH_lh,G_and_S_cingul-Mid-Ant_TH_lh,G_and_S_cingul-Mid-Post_TH_lh,G_cingul-Post-dorsal_TH_lh,G_cingul-Post-ventral_TH_lh,...,S_parieto_occipital_TH_rh,S_pericallosal_TH_rh,S_postcentral_TH_rh,S_precentral-inf-part_TH_rh,S_precentral-sup-part_TH_rh,S_suborbital_TH_rh,S_subparietal_TH_rh,S_temporal_inf_TH_rh,S_temporal_sup_TH_rh,S_temporal_transverse_TH_rh
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,...,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.00000,143.000000
mean,2.069462,2.243112,2.133825,2.404769,2.355056,2.439280,2.423490,2.331692,2.768000,2.113336,...,1.908979,2.435364,1.818832,2.152860,2.061657,2.278923,2.054643,2.156196,2.13651,1.970483
std,0.189288,0.225751,0.192844,0.182563,0.210286,0.203912,0.205958,0.156330,0.231382,0.287778,...,0.151877,0.302305,0.145345,0.155362,0.169625,0.499660,0.199281,0.255395,0.14085,0.325659
min,1.617000,1.780000,1.598000,1.850000,1.785000,1.920000,1.868000,1.914000,1.792000,1.380000,...,1.343000,1.613000,1.422000,1.622000,1.464000,1.421000,1.573000,1.573000,1.73700,1.319000
25%,1.942000,2.079500,2.011000,2.284500,2.218500,2.311000,2.334000,2.229500,2.616500,1.923000,...,1.809000,2.209500,1.718500,2.067500,1.941500,1.956000,1.910000,1.971500,2.02750,1.715500
50%,2.045000,2.239000,2.123000,2.420000,2.348000,2.432000,2.436000,2.330000,2.787000,2.094000,...,1.899000,2.470000,1.813000,2.151000,2.074000,2.177000,2.056000,2.129000,2.13800,1.895000
75%,2.162500,2.383000,2.275500,2.516500,2.495500,2.569000,2.556000,2.435000,2.912000,2.282000,...,2.004500,2.623000,1.922000,2.245000,2.158500,2.521000,2.168000,2.364000,2.23750,2.209500
max,2.729000,2.872000,2.639000,2.800000,2.984000,3.010000,2.972000,2.889000,3.447000,3.015000,...,2.399000,3.274000,2.144000,2.683000,2.597000,4.421000,2.643000,2.808000,2.46100,3.042000


In [192]:
from sklearn import preprocessing

# Mapping to convert labels
DROP_MCI = True # Whether to drop MCI samples or not
fuse_maps = {"SMC": "CN", "EMCI":"MCI", "LMCI":"MCI"}

# Lambda fucntion to be used with Map func
fuse = lambda x: fuse_maps[x] if x in fuse_maps else x
dist = lambda x: pd.Series(x).value_counts()/len(x)

fused_labels = pd.Series(list(map(fuse, raw_labels)))
pd.Series(raw_labels).value_counts()

features, labels = remove_label(raw_features, fused_labels) if DROP_MCI else (raw_features, fused_labels)
print("Sample Size:", len(labels))

le = preprocessing.LabelEncoder()
labels_enc = le.fit_transform(labels)


dist(labels)

Sample Size: 76


CN    0.657895
AD    0.342105
dtype: float64

In [200]:

# Get split returns a generator
# List comprehension is one way to evaluate a generator
X_train, y_train, X_test, y_test = list(get_split(features, labels_enc))[0]
print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)

Train Size: (60, 148)
Test Size: (16,)


In [201]:
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("SVM", SVC(kernel= "rbf", gamma= "scale", C=1))
])

In [202]:
X,y = SMOTE(random_state=42).fit_resample(X_train, y_train)

%time svm_clf.fit(X, y)

CPU times: user 3.41 ms, sys: 744 µs, total: 4.15 ms
Wall time: 3.44 ms


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('SVM', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [203]:
svm_clf.score(X_test, y_test)

0.875

In [204]:
print("Categories:", le.classes_)

Categories: ['AD' 'CN']


In [205]:
y_true = y_test
y_pred = svm_clf.predict(X_test)

plot_confusion_matrix(y_true, y_pred, classes=le.classes_)

Confusion matrix, without normalization
[[5 0]
 [2 9]]


FigureCanvasNbAgg()

## KFold

In [198]:
from sklearn.model_selection import StratifiedKFold as KFold

def getKF(X,y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42 ) #Default = 10

    for train_index, test_index in kf.split(X,y):
        X_train = X.iloc[train_index]
        y_train = y[train_index]
        X_test = X.iloc[test_index]
        y_test = y[test_index]
        
        yield X_train, y_train, X_test, y_test, test_index

scores = 0
testing_indxs =[]
predictions = []
true_labels = []
zoo = []

for X_train, y_train, X_test, y_test, test_index in getKF(features, labels_enc):
    
    X,y = SMOTE(random_state=42).fit_resample(X_train, y_train)
    
    
    svm_clf = Pipeline([
            ("scaler", StandardScaler()),
            ("SVM", SVC(kernel= "rbf", gamma= "scale", C=1))])
    
    
    svm_clf.fit(X, y)
    
    acc = svm_clf.score(X_test, y_test)
    scores += acc
    
    # Updating all information arrays
    testing_indxs.append(test_index)
#     zoo.append(dnn)
    
    y_true = y_test
    y_pred = svm_clf.predict(X_test)
    
    predictions.extend(y_pred)
    true_labels.extend(y_true)
    
    print("Scores on test set: accuracy={:.4f}".format(acc))
print("Average: {:.3f}".format(scores/10))

Scores on test set: accuracy=0.8750
Scores on test set: accuracy=0.7500
Scores on test set: accuracy=0.7500
Scores on test set: accuracy=0.7500
Scores on test set: accuracy=0.6250
Scores on test set: accuracy=0.7500
Scores on test set: accuracy=0.5714
Scores on test set: accuracy=1.0000
Scores on test set: accuracy=0.8571
Scores on test set: accuracy=0.7143
Average: 0.764


In [199]:
plot_confusion_matrix(predictions, true_labels, classes=le.classes_)

Confusion matrix, without normalization
[[16  8]
 [10 42]]


FigureCanvasNbAgg()

In [195]:
plot_confusion_matrix(predictions, true_labels, classes=le.classes_, normalize=True)

Normalized confusion matrix
[[0.66666667 0.33333333]
 [0.19230769 0.80769231]]


FigureCanvasNbAgg()